# Аналіз частоти слів на новинних сайтах з використанням Selenium (7 балів)

## Мета Лабораторної Роботи:
Розвинути навички використання Selenium для збору даних з веб-сторінок.
Практично застосувати Python для аналізу текстових даних і виявлення тенденцій.

## Завдання Лабораторної Роботи:
### Опис Завдання:
Створити скрипт на Python, який використовує Selenium для аналізу частоти зустрічання слів на обраних новинних сайтах за різні місяці та роки.

### Етапи Виконання:
1. Вибір Сайтів для Аналізу: Оберіть кілька новинних сайтів (наприклад, bbc.com, cnn.com).
2. Розробка Скрипта:
    - Ініціалізуйте WebDriver для керування браузером.
    - Налаштуйте скрипт для відвідування архівних розділів обраних сайтів по місяцям та рокам.
    - Зберіть тексти новин за кожен період.
    - Проаналізуйте тексти, визначаючи частоту кожного слова.
3. Аналіз Даних:
    - Порівняйте найбільш часто зустрічаючіся слова в різні місяці та роки.
    - Визначте, які слова були популярні у певні періоди часу.
4. Висновок Результатів:
    - Візуалізуйте результати у вигляді графіків або таблиць.
    - Сформулюйте висновки про динаміку популярності слів у новинній сфері.

## Особливі Вказівки:
- Враховуйте виключно зміст новинних статей, виключаючи коментарі, рекламу та неновинні матеріали.
- Забезпечте коректну обробку тексту, включаючи нормалізацію та очищення від знаків пунктуації та спеціальних символів.


In [19]:
from selenium import webdriver
import time
import pandas as pd

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException

driver = webdriver.Chrome()

driver.get('https://varta1.com.ua/archive/')
time.sleep(1)

df = pd.DataFrame({'Date': [], 'Text': []})

def update_df(date, text):
    df.loc[len(df)] = {'Date': date, 'Text': f'{text}'}
    df.to_csv('archives.csv', index=False)

dates_to_visit = []
dates = []

format_link = lambda year, month, day: f'https://varta1.com.ua/archive/{year}-{month}-{day}/'

def get_data(date_to_visit, date):
    try:
        articles = []
        driver.get(date_to_visit)
        articles = driver.find_elements(By.CSS_SELECTOR, "article.post > a[href]")

        for article in articles:
            try:
                href = article.get_attribute('href')
                driver.get(href)
                main = driver.find_elements(By.CSS_SELECTOR, "main p")
                text = [p.text for p in main]
                driver.back()
                update_df(date, text)
                
            except StaleElementReferenceException:
                pass
            
    except NoSuchElementException:
        pass


for year in [2020, 2021, 2022, 2023]:
    for month in [7]:
        for day in [7]:
            dates_to_visit.append(format_link(year, month, day))
            dates.append(f'{year}-{month}-{day}')

for i in range(len(dates_to_visit)):
    get_data(dates_to_visit[i], dates[i])

time.sleep(5)
